In [1]:
import platform
print(platform.python_version())

3.7.1


In [55]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import to_timestamp, unix_timestamp, max as max_dttm
from pyspark.sql.types import *
spark = SparkSession.builder.appName('PySpark').getOrCreate()
print(spark.version)

2.4.0


In [3]:
df = spark.read.csv('chicago_crimes_2001_to_present.csv', header=True, inferSchema=True)
# df.show(2)
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [4]:
df = df.withColumn('dttm', unix_timestamp('Date', "MM/dd/yyyy hh:mm:ss aa").cast(TimestampType()))

In [38]:
# df.printSchema()
# df.groupBy("ID", "Case Number").agg(max_dttm("dttm")).show()

# df.groupby().max('dttm').collect()[0].asDict()['max(dttm)']
# df.groupBy("Date").agg(max("dttm")).show()

print(df.agg({"dttm":"max"}).collect()[0][0])
print(df.agg({"dttm":"min"}).collect()[0][0])


2018-11-22 23:56:00
2001-01-01 00:00:00
1225414555.8439384


In [52]:
from pyspark.sql.functions import year
year_general = df.select(year('dttm').alias('year')).groupBy('year').count().orderBy('year')
year_general.show()
# year_general_max = year_general.agg({"year":"max"}).collect()[0][0]
# print(year_general_max)
# year_general.filter(year_general['count'] == year_general_max).show()
# print(year_general.agg({"year":"min"}).collect()[0][0])
# print(year_general.agg({"year":"avg"}).collect()[0][0])

+----+------+
|year| count|
+----+------+
|null|   247|
|2001|485738|
|2002|486742|
|2003|475923|
|2004|469366|
|2005|453686|
|2006|448078|
|2007|436986|
|2008|427024|
|2009|392649|
|2010|370277|
|2011|351736|
|2012|335908|
|2013|307020|
|2014|275196|
|2015|263930|
|2016|268814|
|2017|267811|
|2018|237148|
+----+------+



In [57]:
max_value = year_general.agg({"count":"max"}).collect()[0][0]
max_year = year_general.filter(year_general['count'] == max_value).collect()[0][0]
print(str(max_year) + ": " + str(max_value))

2002: 486742
